In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import os
os.chdir('/Users/steve/GetOldTweets3-0.0.10')
import re
import nltk
import contractions

os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

from sent2vec.vectorizer import Vectorizer

from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!python3 setup.py install
import fasttext.util

from imblearn.combine import SMOTETomek

from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, strip_tags, strip_punctuation, stem_text, preprocess_documents, strip_multiple_whitespaces, strip_non_alphanum, strip_short

fatal: destination path 'fastText' already exists and is not an empty directory.
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [3]:
nltk.download('words')
words = set(nltk.corpus.words.words())
additional_words = ('joebiden', 'biden', 'donaldtrump', 'trump', 'barackobama', 'obama')
for ele in additional_words:
    words.add(ele)

[nltk_data] Downloading package words to /Users/steve/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_non_alphanum, stem_text,
                  remove_stopwords, strip_short]

def prep(sentence): 
    sentence = contractions.fix(sentence)
    split_sentence = sentence.split()
    return preprocess_string(sentence, CUSTOM_FILTERS)

def prep_withspell(sentence):
    holder = " "
    sentence = contractions.fix(sentence)
    preprocessed = preprocess_string(sentence, CUSTOM_FILTERS)
    spell_correct = TextBlob(holder.join(preprocessed))
    corrected_sentence = spell_correct.correct()
    return corrected_sentence.split()    

def prepare(sentence): 
    processed_feature = re.sub(r'\W', ' ', str(sentence))
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    processed_feature = re.sub(r'^b\s+', '', processed_feature)
    processed_feature = processed_feature.lower()
    stop_words = set(stopwords.words('english'))
    sentence = processed_feature.split()
    removed_stopwords = [w for w in sentence if not w.lower() in stop_words]    
    return ' '.join(removed_stopwords)

In [5]:
def cont_to_multiclass(cont):
    if cont > 0.66:
        return 'highly positive'
    elif cont > 0.33:
        return 'positive'
    elif cont > 0:
        return 'partly positive'
    elif cont > -0.33:
        return 'partly negative'
    elif cont > -0.66:
        return 'negative'
    else:
        return 'highly negative'

def cont_to_binary(cont):
    if cont > 0:
        #return 'positive'
        return 1
    else:
        #return 'negative'
        return 0

In [6]:
def get_document_frequency(data, wi, wj=None):
    if wj is None:
        D_wi = 0
        for l in range(len(data)):
            doc = data[l]
            if wi in doc:
                D_wi += 1
        return D_wi
    D_wj = 0
    D_wi_wj = 0
    for l in range(len(data)):
        doc = data[l]
        if wj in doc:
            D_wj += 1
            if wi in doc:
                D_wi_wj += 1
    return D_wj, D_wi_wj

def get_topic_coherence(beta, data, vocab, seed):
    D = len(data)
    TC = []
    num_topics = len(beta.components_)
    selected = -1
    selected = []
    for k, topic in enumerate(beta.components_):
        print('k: {}/{}'.format(k, num_topics))
        top_10 = topic.argsort()[:-20 - 1:-1]
        top_words = [vocab[i] for i in top_10]
        print(top_words)
        TC_k = 0
        counter = 0
        for i, word in enumerate(top_words):
            D_wi = get_document_frequency(data, word)
            j = i + 1
            tmp = 0
            while j < len(top_10) and j > i:
                D_wj, D_wi_wj = get_document_frequency(data, word, top_words[j])
                if D_wi_wj == 0:
                    f_wi_wj = -1
                else:
                    f_wi_wj = -1 + ( np.log(D_wi) + np.log(D_wj)  - 2.0 * np.log(D) ) / ( np.log(D_wi_wj) - np.log(D) )
                tmp += f_wi_wj
                j += 1
                counter += 1
            TC_k += tmp 
        TC.append(TC_k)
    print('num topics: ', len(TC))
    print('Topic Coherence is: {}'.format(TC))
    return TC, selected

In [7]:
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

In [8]:
def word_vector(model, tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [9]:
key_words = ['leftwing', 'rightwing', 'liberal', 'leftist', 'conservative', 'politics', 'political', 'antifa', 'gop',
            'democrat', 'trump', 'maga', 'republican', 'politicians', 'vote', 'government', 'breakingnews', 'worldnews', 
            'news', 'fakenews', 'obama', 'tcot', 'p2']

key_word_test = ['leftwing', 'rightwing', 'liberal']

leftwing_df = pd.read_csv('leftwing.csv', error_bad_lines=False, header=None, index_col=False, sep="<|>", lineterminator='\n')
rightwing_df = pd.read_csv('rightwing.csv', error_bad_lines=False, header=None, index_col=False, sep="<|>", lineterminator='\n')
liberal_df = pd.read_csv('liberal.csv', error_bad_lines=False, header=None, index_col=False, sep= '<|>', lineterminator='\n')
#leftist_df = pd.read_csv('leftist.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#conservative_df = pd.read_csv('conservative.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#politics_df = pd.read_csv('politics.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#political_df = pd.read_csv('political.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#gop_df = pd.read_csv('gop.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#democrat_df = pd.read_csv('democrat.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#trump_df = pd.read_csv('trump.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#republican_df = pd.read_csv('republican.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#politicians_df = pd.read_csv('politicians.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#fakenews = pd.read_csv('obama.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#tcot = pd.read_csv('tcot.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')
#p2 = pd.read_csv('p2.csv', error_bad_lines=False, header=None, index_col=False, sep='<|>', lineterminator='\n')

#frames = [leftwing_df, rightwing_df, liberal_df, leftist_df, conservative_df, politics_df, political_df, antifa_df, gop_df, democrat_df, trump_df, republican_df, politicians_df, vote_df, government_df, breakingnews_df, worldnews, news, fakenews, obama, tcot, p2]
#dataset = pd.concat(frames, keys=key_words)

frames = [leftwing_df, rightwing_df, liberal_df]
dataset = pd.concat(frames, keys=key_word_test)
dataset = dataset.drop_duplicates()
holder = dataset[dataset[0].apply(lambda row: len(row.split()) < 4)]
dataset = pd.concat((dataset, holder))
dataset = dataset.drop_duplicates(keep=False)
dataset[3] = dataset[2].apply(lambda row: str(row).split()[:6])
dataset[3] = dataset[3].apply(lambda row: ' '.join([str(elem) for elem in row]))
dataset = dataset.drop_duplicates(subset=[1, 3], keep=False)

/Users/steve/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
/Users/steve/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
dataset[0] = dataset.apply(lambda row: row[0].split()[1], axis=1)                                                
dataset[2] = dataset.apply(lambda row: re.sub(r'http\S+', '', str(row[2])), axis=1)
dataset[2] = dataset.apply(lambda row: ''.join([c for c in row[2] if not c.isdigit()]), axis=1)
dataset[3] = dataset.apply(lambda row : " ".join(w for w in nltk.wordpunct_tokenize(row[2]) if w.lower() in words or not w.isalpha()), axis = 1)
dataset[4] = dataset.apply(lambda row : prep(row[3]), axis = 1)
dataset = dataset[dataset[4].map(len) >= 5].sort_values([0], ascending=[True])

processed_data = [" ".join(x) for x in dataset[4]]

In [10]:
def replace(sent):
    sent = sent.replace('donaldtrump', 'trump')
    sent = sent.replace('new', 'news')
    sent = sent.replace('becaus', 'because')
    sent = sent.replace('extrem', 'extreme')
    sent = sent.replace('liber', 'liberal')
    sent = sent.replace('peopl', 'people')
    sent = sent.replace('govern', 'government')
    return sent.replace('femin', 'feminist')

In [11]:
for index, sent in enumerate(processed_data):
    processed_data[index] = replace(sent)

In [12]:
tfidf = TfidfVectorizer(max_df=0.90, min_df=50, stop_words='english', ngram_range=(1,3))
dtm = tfidf.fit_transform(processed_data)

nmf_model = NMF(n_components=50, random_state=42, beta_loss='kullback-leibler', solver='mu', 
                max_iter=1000, alpha=2, l1_ratio=0.5)
nmf_topics = nmf_model.fit_transform(dtm)

for index, topic in enumerate(nmf_model.components_):
    print(f'THE TOP 50 WORDS FOR TOPIC #{index} GIVEN BY NMF:')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-50:]])
    print('\n')

/Users/steve/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


THE TOP 50 WORDS FOR TOPIC #0 GIVEN BY NMF:
['plai', 'went', 'data', 'warn', 'servic', 'light', 'realiti', 'histor', 'evid', 'check', 'gave', 'reveal', 'recent', 'walk', 'posit', 'seriou', 'ground', 'late', 'offic', 'train', 'target', 'tri', 'fals', 'intellig', 'secret', 'remain', 'miss', 'beat', 'refus', 'write', 'intern', 'came', 'threat', 'establish', 'clear', 'remov', 'biggest', 'past', 'issu', 'close', 'game', 'base', 'set', 'danger', 'hold', 'head', 'mai', 'forc', 'leav', 'face']


THE TOP 50 WORDS FOR TOPIC #1 GIVEN BY NMF:
['frank', 'ethic', 'molli', 'earli', 'ann', 'march', 'probe', 'deal', 'round', 'van', 'nanci', 'cross', 'secretari', 'pundit', 'kim', 'mari', 'inaugur', 'sen', 'chief', 'laura', 'warren', 'investig', 'eric', 'trump administr', 'hill', 'border', 'cartoon', 'attornei', 'administr', 'repeal', 'jeff', 'session', 'travel', 'ben', 'sam', 'confirm', 'nick', 'peter', 'lee', 'rep', 'king', 'ban', 'pick', 'gener', 'mark', 'north', 'trump', 'progress liberal', 'liberal 

In [13]:
tf_vectorizer = CountVectorizer(analyzer='word', min_df=50, ngram_range=(1, 3))
tf_fit = tf_vectorizer.fit_transform(processed_data)

lda_model = LatentDirichletAllocation(n_components=50, doc_topic_prior=.01)
lda_topics = lda_model.fit_transform(tf_fit)

for index, topic in enumerate(lda_model.components_):
    print(f'THE TOP 50 WORDS FOR TOPIC #{index} GIVEN BY LDA:')
    print([tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-50:]])
    print('\n')

THE TOP 50 WORDS FOR TOPIC #0 GIVEN BY LDA:
['death', 'threat liberal', 'raw', 'kamala', 'trump biden', 'loan', 'michigan', 'sen', 'lead', 'trump polit democrat', 'thi liberal democrat', 'race liberal', 'statu quo', 'democrat polit', 'liberal democrat trump', 'contact', 'liberal power', 'victori', 'threat', 'debat polit', 'presidenti debat', 'quo', 'dump', 'democrat trump', 'grab', 'liberal said', 'snowflak liberal', 'power liberal', 'transit', 'elect', 'race', 'liberal snowflak', 'statu', 'presidenti', 'warren', 'power', 'debat', 'sander', 'biden', 'snowflak', 'readi polit democrat', 'readi polit', 'readi', 'trump', 'democrat liberal', 'polit', 'polit democrat', 'liberal', 'liberal democrat', 'democrat']


THE TOP 50 WORDS FOR TOPIC #1 GIVEN BY LDA:
['wast time', 'try', 'hypocrit liberal', 'job', 'free liberal', 'school', 'liberal liar', 'realli', 'liberal free', 'good thing', 'like', 'high school', 'onli', 'rag', 'speech liberal', 'liberal let', 'good luck', 'hard work', 'liberal goo

In [14]:
naming={0:'Topic0', 1:'Topic1', 2:'Topic2', 3:'Topic3', 4:'Topic4', 5:'Topic5', 6:'Topic6', 7:'Topic7',
        8:'Topic8', 9:'Topic9', 10:'Topic10', 11:'Topic11', 12:'Topic12', 13:'Topic13', 14:'Topic14', 15:'Topic15',
        16:'Topic16', 17:'Topic17', 18:'Topic18', 19:'Topic19', 20:'Topic20', 21:'Topic21', 22:'Topic22', 23:'Topic23',
        24:'Topic24', 25:'Topic25', 26:'Topic26', 27:'Topic27', 28:'Topic28', 29:'Topic29', 30:'Topic30', 31:'Topic31',
        32:'Topic32', 33:'Topic33', 34:'Topic34', 35:'Topic35', 36:'Topic36', 37:'Topic37', 38:'Topic38', 39:'Topic39',
        40:'Topic40', 41:'Topic41', 42:'Topic42', 43:'Topic43', 44:'Topic44', 45:'Topic45', 46:'Topic46', 47:'Topic47',
        48:'Topic48', 49:'Topic49'}


keys = {'leftwing', 'rightwing'}
dataset[5] = nmf_topics.argmax(axis=1)
dataset[5] = dataset[5].map(naming)
dataset[6] = lda_topics.argmax(axis=1)
dataset[6] = dataset[6].map(naming)
dataset[7] = dataset[4].apply(lambda row: ' '.join(row))
dataset[8] = dataset[2].apply(lambda row: re.findall(r"#(\w+)", row))
dataset[8] = dataset[8].apply(lambda row: [each_string.lower() for each_string in row])
dataset[8] = dataset[8].apply(lambda row: list(set(row).difference(keys)))

In [15]:
def replace_hashtags(used_hashtags):
    for ele in used_hashtags:
        if ele == 'trump':
            ele = 'donald_trump'
    return used_hashtags
dataset[8] = dataset[8].apply(lambda row: [] if len(row)<2 else row)
dataset[8] = dataset[8].apply(lambda row: replace_hashtags(row))

In [16]:
lab_num = 20000
labeled_sentiment = dataset.sample(n=lab_num)
labeled_sentiment[9] = labeled_sentiment.apply(lambda row : sentiment_scores(row[2]), axis = 1)
labeled_sentiment[9] = labeled_sentiment[9].apply(lambda row: cont_to_binary(row))
dataset = dataset.join(labeled_sentiment[9], how='left', lsuffix='_left', rsuffix='_right')

In [17]:
import warnings
warnings.filterwarnings("ignore")
pd.set_option("max_columns", 30)
import itertools
import string

def listOccurences(item, names):
    set_of_items = set(item)
    count = 0
    for value in names:
        if set_of_items & set(value)==set_of_items:
            count += 1
    return count

hashtags =  list(dataset[8][:1000])
permuted_values = list(itertools.combinations(set(itertools.chain.from_iterable(hashtags)), 2))
d = {}
for v in permuted_values:
    d[str(v)] = listOccurences(v, hashtags)

hashtag_data = {i: j for i, j in d.items() if j >= 5}
occurence_count = pd.DataFrame.from_dict(hashtag_data, orient='index')
occurence_count = occurence_count.reset_index()
holder = occurence_count['index'].str.split(", ", n=1, expand=True)
occurence_count["Element1"] = holder[0]
occurence_count["Element2"] = holder[1]
occurence_count.drop(columns=["index"], inplace = True)
occurence_count['Element1'] = occurence_count['Element1'].apply(lambda row: row.translate(str.maketrans('', '', string.punctuation)))
occurence_count['Element2'] = occurence_count['Element2'].apply(lambda row: row.translate(str.maketrans('', '', string.punctuation)))

In [18]:
occurence_count.to_csv(os.getcwd()+'\\occurence_count.csv', index = False)

In [19]:
%%capture
fasttext.util.download_model('en', if_exists='ignore')
ft_model = fasttext.load_model('cc.en.300.bin')
ft = pd.DataFrame(processed_data, columns=['tweets'])
ft['tweets'] = ft['tweets'].apply(lambda row: ft_model.get_sentence_vector(row))
ft = np.stack(ft['tweets'].to_numpy())

In [20]:
embedder = sister.MeanEmbedding(lang='en')
s2v = pd.DataFrame(processed_data, columns=['tweets'])
s2v['tweets'] = s2v['tweets'].apply(lambda row: embedder(row))
s2v = np.stack(s2v['tweets'].to_numpy())

Loading model...


In [ ]:
dtm_ar = dtm.toarray()
tf_fit_ar = tf_fit.toarray()
embed_array = np.hstack((dtm_ar, tf_fit_ar, ft, s2v))
embed_array_df = pd.DataFrame(embed_array, index=dataset.index)
labeled_sentiment_embeddings = labeled_sentiment.join(embed_array_df, how='left', lsuffix='_left', rsuffix='_right')
labeled_sentiment_embeddings.drop(['0_left', '1_left', '2_left', '3_left', '4_left', '5_left'], axis=1, inplace=True)
labeled_sentiment_embeddings.drop(['6_left','7_left', '8_left', '9_left'], axis=1, inplace=True)
print('here')

smt = SMOTETomek(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(labeled_sentiment_embeddings, labeled_sentiment[9], test_size=0.1, random_state=42)
X_train, y_train = smt.fit_resample(X_train, y_train)

In [ ]:
X_train_dtm = X_train[X_train.columns[0:dtm.shape[1]]]
X_train_tf = X_train[X_train.columns[dtm.shape[1]:(dtm.shape[1]+tf_fit.shape[1])]]
X_train_tf = X_train[X_train.columns[dtm.shape[1]:(dtm.shape[1]+tf_fit.shape[1])]]
X_train_ft = X_train[X_train.columns[(dtm.shape[1]+tf_fit.shape[1]):(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1])]]
X_train_s2v = X_train[X_train.columns[(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1]):(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1]+s2v.shape[1])]]
X_test_dtm = X_test[X_test.columns[0:dtm.shape[1]]]
X_test_tf = X_test[X_test.columns[dtm.shape[1]:(dtm.shape[1]+tf_fit.shape[1])]]
X_test_ft = X_test[X_test.columns[(dtm.shape[1]+tf_fit.shape[1]):(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1])]]
X_test_s2v = X_test[X_test.columns[(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1]):(dtm.shape[1]+tf_fit.shape[1]+ft.shape[1]+s2v.shape[1])]]
dtm_df = pd.DataFrame(dtm_ar, columns=X_test_dtm.columns)
tf_fit_df = pd.DataFrame(tf_fit_ar, columns=X_test_tf.columns)
ft_df = pd.DataFrame(ft, columns=X_test_ft.columns)
s2v_df = pd.DataFrame(s2v, columns=X_test_s2v.columns)

In [ ]:
lab_num=20000
lab_num=int(lab_num/10)
rf_pred = pd.DataFrame(np.empty((lab_num, 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
etc_pred = pd.DataFrame(np.empty((lab_num, 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
sgd_pred = pd.DataFrame(np.empty((lab_num, 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
xgb_pred = pd.DataFrame(np.empty((lab_num, 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])

rf_pred_full = pd.DataFrame(np.empty((len(dataset[0]), 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
etc_pred_full = pd.DataFrame(np.empty((len(dataset[0]), 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
sgd_pred_full = pd.DataFrame(np.empty((len(dataset[0]), 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])
xgb_pred_full = pd.DataFrame(np.empty((len(dataset[0]), 4)), columns=['dtm', 'tf_fit', 'fasttext', 's2v'])

In [ ]:
print("RandomForest: ")
rf_classifier_dtm = RandomForestClassifier(n_estimators=300, max_depth=300, random_state=42)
rf_classifier_dtm.fit(X_train_dtm, y_train)
predictions = rf_classifier_dtm.predict(X_test_dtm)
rf_pred['dtm'] = predictions
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
predictions = rf_classifier_dtm.predict(dtm_df)
rf_pred_full['dtm'] = predictions

print("ExtraTree: ")
etc_dtm = ExtraTreesClassifier(n_estimators=300, random_state=42)
etc_dtm.fit(X_train_dtm, y_train)
predictions = etc_dtm.predict(X_test_dtm)
etc_pred['dtm'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = etc_dtm.predict(dtm_df)
etc_pred_full['dtm'] = predictions

print("StochasticGradientDescent: ")
sgd_classifier_dtm = SGDClassifier(loss="hinge", penalty="l1")
sgd_classifier_dtm.fit(X_train_dtm, y_train)
predictions = sgd_classifier_dtm.predict(X_test_dtm)
sgd_pred['dtm'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = sgd_classifier_dtm.predict(dtm_df)
sgd_pred_full['dtm'] = predictions

print("XGBoost: ")
xgb_classifier_dtm = XGBClassifier(n_estimator=200, learning_rate=0.1, eval_metric='mlogloss', random_state=42)
xgb_classifier_dtm.fit(X_train_dtm, y_train)
predictions = xgb_classifier_dtm.predict(X_test_dtm)
xgb_pred['dtm'] = predictions
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))
predictions = xgb_classifier_dtm.predict(dtm_df)
xgb_pred_full['dtm'] = predictions

In [ ]:
print("RandomForest: ")
rf_classifier_tf = RandomForestClassifier(n_estimators=300, max_depth=300, random_state=42)
rf_classifier_tf.fit(X_train_tf, y_train)
predictions = rf_classifier_tf.predict(X_test_tf)
rf_pred['tf_fit'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = rf_classifier_tf.predict(tf_fit)
rf_pred_full['tf_fit'] = predictions


print("ExtraTree: ")
etc_tf = ExtraTreesClassifier(n_estimators=300, random_state=42)
etc_tf.fit(X_train_tf, y_train)
predictions = etc_tf.predict(X_test_tf)
etc_pred['tf_fit'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = etc_tf.predict(tf_fit)
etc_pred_full['tf_fit'] = predictions

print("StochasticGradientDescent: ")
sgd_classifier_tf = SGDClassifier(loss="hinge", penalty="l1")
sgd_classifier_tf.fit(X_train_tf, y_train)
predictions = sgd_classifier_tf.predict(X_test_tf)
sgd_pred['tf_fit'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = sgd_classifier_tf.predict(tf_fit)
sgd_pred_full['tf_fit'] = predictions

print("XGBoost: ")
xgb_classifier_tf = XGBClassifier(n_estimator=200, learning_rate=0.1, eval_metric='mlogloss', random_state=42)
xgb_classifier_tf.fit(X_train_tf, y_train)
predictions = xgb_classifier_tf.predict(X_test_tf)
xgb_pred['tf_fit'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = xgb_classifier_tf.predict(tf_fit_df)
xgb_pred_full['tf_fit'] = predictions

In [ ]:
print("RandomForest: ")
rf_classifier_ft = RandomForestClassifier(n_estimators=300, max_depth=300, random_state=42)
rf_classifier_ft.fit(X_train_ft, y_train)
predictions = rf_classifier_ft.predict(X_test_ft)
rf_pred['fasttext'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = rf_classifier_ft.predict(ft)
rf_pred_full['fasttext'] = predictions

print("ExtraTree: ")
etc_ft = ExtraTreesClassifier(n_estimators=300, random_state=42)
etc_ft.fit(X_train_ft, y_train)
predictions = etc_ft.predict(X_test_ft)
etc_pred['fasttext'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = etc_ft.predict(ft)
etc_pred_full['fasttext'] = predictions

print("StochasticGradientDescent: ")
sgd_classifier_ft = SGDClassifier(loss="hinge", penalty="l1")
sgd_classifier_ft.fit(X_train_ft, y_train)
predictions = sgd_classifier_ft.predict(X_test_ft)
sgd_pred['fasttext'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = sgd_classifier_ft.predict(ft)
sgd_pred_full['fasttext'] = predictions

print("XGBoost: ")
xgb_classifier_ft = XGBClassifier(n_estimator=200, learning_rate=0.1, eval_metric='mlogloss', random_state=42)
xgb_classifier_ft.fit(X_train_ft, y_train)
predictions = xgb_classifier_ft.predict(X_test_ft)
xgb_pred['fasttext'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = xgb_classifier_ft.predict(ft_df)
xgb_pred_full['fasttext'] = predictions

In [ ]:
print("RandomForest: ")
rf_classifier_s2v = RandomForestClassifier(n_estimators=300, max_depth=300, random_state=42)
rf_classifier_s2v.fit(X_train_s2v, y_train)
predictions = rf_classifier_s2v.predict(X_test_s2v)
rf_pred['s2v'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = rf_classifier_s2v.predict(s2v)
rf_pred_full['s2v'] = predictions

print("ExtraTree: ")
etc_s2v = ExtraTreesClassifier(n_estimators=300, random_state=42)
etc_s2v.fit(X_train_s2v, y_train)
predictions = etc_s2v.predict(X_test_s2v)
etc_pred['s2v'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = etc_s2v.predict(s2v)
etc_pred_full['s2v'] = predictions

print("StochasticGradientDescent: ")
sgd_classifier_s2v = SGDClassifier(loss="hinge", penalty="l1")
sgd_classifier_s2v.fit(X_train_s2v, y_train)
predictions = sgd_classifier_s2v.predict(X_test_s2v)
sgd_pred['s2v'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = sgd_classifier_s2v.predict(s2v)
sgd_pred_full['s2v'] = predictions

print("XGBoost: ")
xgb_classifier_s2v = XGBClassifier(n_estimator=200, learning_rate=0.1, eval_metric='mlogloss', random_state=42)
xgb_classifier_s2v.fit(X_train_s2v, y_train)
predictions = xgb_classifier_s2v.predict(X_test_s2v)
xgb_pred['s2v'] = predictions
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))
predictions = xgb_classifier_s2v.predict(s2v_df)
xgb_pred_full['s2v'] = predictions

In [ ]:
rf_pred = (rf_pred.fillna(0)).replace([np.inf, -np.inf], 0).replace([np.inf, -np.inf], 0).astype('float64')
rf_pred_full = (rf_pred_full.fillna(0)).replace([np.inf, -np.inf], 0).astype('float64')
etc_pred = (etc_pred.fillna(0)).replace([np.inf, -np.inf], np.nan).astype('float64')
etc_pred_full = (etc_pred_full.fillna(0)).replace([np.inf, -np.inf], 0).astype('float64')
sgd_pred = (sgd_pred.fillna(0)).replace([np.inf, -np.inf], np.nan).astype('float64')
sgd_pred_full = (sgd_pred_full.fillna(0)).replace([np.inf, -np.inf], 0).astype('float64')
xgb_pred = (xgb_pred.fillna(0)).replace([np.inf, -np.inf], 0).astype('float64')
xgb_pred_full = (xgb_pred_full.fillna(0)).replace([np.inf, -np.inf], 0).astype('float64')

In [ ]:
print("RF Model: ")
rf_model = LogisticRegression(random_state=42).fit(rf_pred, y_test)
predictions = rf_model.predict(rf_pred)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

print("ETC Model: ")
etc_model = LogisticRegression(random_state=42).fit(etc_pred, y_test)
predictions = etc_model.predict(etc_pred)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

print("SGD Model: ")
sgd_model = LogisticRegression(random_state=42).fit(sgd_pred, y_test)
predictions = sgd_model.predict(sgd_pred)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

print("XGB Model: ")
xgb_model = LogisticRegression(random_state=42).fit(xgb_pred, y_test)
predictions = xgb_model.predict(xgb_pred)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

In [ ]:
dataset[10] = rf_model.predict(rf_pred_full)
dataset[11] = etc_model.predict(etc_pred_full)
dataset[12] = sgd_model.predict(sgd_pred_full)
dataset[13] = xgb_model.predict(xgb_pred_full)

sent_pred = dataset[9]
ml_pred = dataset[11]
sent_pred[np.isnan(sent_pred)] = ml_pred
dataset[14] = sent_pred.to_frame()

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
stopwords = set(STOPWORDS)
keys = ['polit', 'thi', 'thei', 'add', 'onli']
for ele in keys:
    stopwords.add(ele)

text = ' '.join(processed_data)
flame_mask = np.array(Image.open("/Users/steve/NLP-2021-SummerLab/Images/flame.jpg"))
mask_mask = np.array(Image.open("/Users/steve/NLP-2021-SummerLab/Images/mask.jpg"))
resist_mask = np.array(Image.open("/Users/steve/NLP-2021-SummerLab/Images/resist.jpg"))
gun_mask = np.array(Image.open("/Users/steve/NLP-2021-SummerLab/Images/gun.jpg"))
tweet_mask = np.array(Image.open("/Users/steve/NLP-2021-SummerLab/Images/twitter_logo.jpg"))

flame_nmf_wc = WordCloud(background_color="white", mask=flame_mask, max_words=2000, stopwords=stopwords, contour_width=3, contour_color='steelblue')
mask_nmf_wc = WordCloud(background_color="white", mask=mask_mask, max_words=2000, stopwords=stopwords, contour_width=3, contour_color='steelblue')
resist_nmf_wc = WordCloud(background_color="white", mask=resist_mask, max_words=2000, stopwords=stopwords, contour_width=3, contour_color='steelblue')
gun_nmf_wc = WordCloud(background_color="white", mask=gun_mask, max_words=2000, stopwords=stopwords, contour_width=3, contour_color='steelblue')
tweet_nmf_wc = WordCloud(background_color="white", mask=tweet_mask, max_words=2000, stopwords=stopwords, contour_width=3, contour_color='steelblue')

flame_nmf_wc.generate(text)
mask_nmf_wc.generate(text)
resist_nmf_wc.generate(text)
gun_nmf_wc.generate(text)
tweet_nmf_wc.generate(text)

plt.imshow(flame_nmf_wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(mask_nmf_wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(resist_nmf_wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(gun_nmf_wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(tweet_nmf_wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.show()

In [ ]:
race_nmf, race_lda = 'Topic28', 'Topic3'
gun_nmf, gun_lda = 'Topic31', 'Topic50'
mask_nmf, mask_lda = 'Topic27', 'Topic5'
resist_nmf, resist_lda = 'Topic25', 'Topic42'
immig_nmf, immig_lda = 'Topic45', 'Topic44'
race_nmf = dataset[dataset[5] == race_nmf]
gun_nmf = dataset[dataset[5] == gun_nmf]
mask_nmf = dataset[dataset[5] == mask_nmf]
resist_nmf = dataset[dataset[5] == resist_nmf]
immig_nmf = dataset[dataset[5] == immig_nmf]

from collections import Counter
race_nmf_vol = pd.DataFrame.from_dict(Counter([date[0:7] for date in (race_nmf[0].values)]), orient='index').reset_index()
gun_nmf_vol = pd.DataFrame.from_dict(Counter([date[0:7] for date in (gun_nmf[0].values)]), orient='index').reset_index()
mask_nmf_vol = pd.DataFrame.from_dict(Counter([date[0:7] for date in (mask_nmf[0].values)]), orient='index').reset_index()
resist_nmf_vol = pd.DataFrame.from_dict(Counter([date[0:7] for date in (resist_nmf[0].values)]), orient='index').reset_index()
immig_nmf_vol = pd.DataFrame.from_dict(Counter([date[0:7] for date in (immig_nmf[0].values)]), orient='index').reset_index()

In [ ]:
import datetime
race_nmf_vol = race_nmf_vol.sort_values(by=['index'])
race_nmf_vol['index'] = [datetime.datetime.strptime(d,"%Y-%m").date() for d in race_nmf_vol['index']]
plt.plot(race_nmf_vol['index'], race_nmf_vol[0])
plt.ylabel('# Posts per Month')
plt.title('Race NMF Volumetric Analysis')
#plt.locator_params(axis="x", nbins=4)
plt.show()

In [ ]:
gun_nmf_vol = gun_nmf_vol.sort_values(by=['index'])
gun_nmf_vol['index'] = [datetime.datetime.strptime(d,"%Y-%m").date() for d in gun_nmf_vol['index']]
plt.plot(gun_nmf_vol['index'], gun_nmf_vol[0])
plt.ylabel('# Posts per Month')
plt.title('Gun NMF Volumetric Analysis')
plt.show()

In [ ]:
mask_nmf_vol = mask_nmf_vol.sort_values(by=['index'])
mask_nmf_vol['index'] = [datetime.datetime.strptime(d,"%Y-%m").date() for d in mask_nmf_vol['index']]
plt.plot(mask_nmf_vol['index'], mask_nmf_vol[0])
plt.ylabel('# Posts per Month')
plt.title('COVID NMF Volumetric Analysis')
#plt.locator_params(axis="x", nbins=4)
plt.show()

In [ ]:
resist_nmf_vol = resist_nmf_vol.sort_values(by=['index'])
resist_nmf_vol['index'] = [datetime.datetime.strptime(d,"%Y-%m").date() for d in resist_nmf_vol['index']]
plt.plot(resist_nmf_vol['index'], resist_nmf_vol[0])
plt.ylabel('# Posts per Month')
plt.title('Resist NMF Volumetric Analysis')
#plt.locator_params(axis="x", nbins=4)
plt.show()

In [ ]:
immig_nmf_vol = immig_nmf_vol.sort_values(by=['index'])
immig_nmf_vol['index'] = [datetime.datetime.strptime(d,"%Y-%m").date() for d in immig_nmf_vol['index']]
plt.plot(immig_nmf_vol['index'], immig_nmf_vol[0])
plt.ylabel('# Posts per Month')
plt.title('Immigration NMF Volumetric Analysis')
plt.locator_params(axis="x", nbins=4)
plt.show()

In [ ]:
race_sentiment = pd.DataFrame([date for date in (race_nmf[14].values)], index=(race_nmf[0].values))
race_sentiment_pos = race_sentiment.groupby(race_sentiment[0]).get_group(1)
race_sentiment_neg = race_sentiment.groupby(race_sentiment[0]).get_group(0)
race_sentiment_pos = (pd.DataFrame.from_dict(Counter(race_sentiment_pos.index.str[:7]), orient='index').reset_index()).sort_values(by=['index'])
race_sentiment_neg = (pd.DataFrame.from_dict(Counter(race_sentiment_neg.index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')

ax = plt.gca()

race_sentiment_pos.plot(style=[':', '--', '-'], color='blue', ax=ax)
race_sentiment_neg.plot(style=[':', '--', '-'], color='red', ax=ax)
plt.ylabel('# Posts per Month')
plt.xlabel('Time-Series')
plt.xticks(np.arange(0, max(len(race_sentiment_pos), len(race_sentiment_neg))+1, 10))
plt.title('Race NMF Sentiment Analysis')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
gun_sentiment = pd.DataFrame([date for date in (gun_nmf[14].values)], index=(gun_nmf[0].values))
gun_sentiment_pos = gun_sentiment.groupby(gun_sentiment[0]).get_group(1)
gun_sentiment_neg = gun_sentiment.groupby(gun_sentiment[0]).get_group(0)
gun_sentiment_pos = (pd.DataFrame.from_dict(Counter(gun_sentiment_pos[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')
gun_sentiment_neg = (pd.DataFrame.from_dict(Counter(gun_sentiment_neg[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')

ax = plt.gca()

gun_sentiment_pos.plot(style=[':', '--', '-'], color='blue', ax=ax)
gun_sentiment_neg.plot(style=[':', '--', '-'], color='red', ax=ax)
plt.ylabel('# Gun per Month')
plt.xlabel('Time-Series')
plt.title('Immigration NMF Sentiment Analysis')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
mask_sentiment = pd.DataFrame([date for date in (mask_nmf[14].values)], index=(mask_nmf[0].values))
mask_sentiment_pos = mask_sentiment.groupby(mask_sentiment[0]).get_group(1)
mask_sentiment_neg = mask_sentiment.groupby(mask_sentiment[0]).get_group(0)
mask_sentiment_pos = (pd.DataFrame.from_dict(Counter(mask_sentiment_pos[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')
mask_sentiment_neg = (pd.DataFrame.from_dict(Counter(mask_sentiment_neg[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')

ax = plt.gca()

mask_sentiment_pos.plot(style=[':', '--', '-'], color='blue', ax=ax)
mask_sentiment_neg.plot(style=[':', '--', '-'], color='red', ax=ax)
plt.ylabel('# Posts per Month')
plt.xlabel('Time-Series')
plt.title('Healthcare NMF Sentiment Analysis')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
resist_sentiment = pd.DataFrame([date for date in (resist_nmf[14].values)], index=(resist_nmf[0].values))
resist_sentiment_pos = resist_sentiment.groupby(resist_sentiment[0]).get_group(1)
resist_sentiment_neg = resist_sentiment.groupby(resist_sentiment[0]).get_group(0)
resist_sentiment_pos = (pd.DataFrame.from_dict(Counter(resist_sentiment_pos[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')
resist_sentiment_neg = (pd.DataFrame.from_dict(Counter(resist_sentiment_neg[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')

ax = plt.gca()

resist_sentiment_pos.plot(style=[':', '--', '-'], color='blue', ax=ax)
resist_sentiment_neg.plot(style=[':', '--', '-'], color='red', ax=ax)
plt.ylabel('# Posts per Month')
plt.xlabel('Time-Series')
plt.title('Racism NMF Sentiment Analysis')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
immig_sentiment = pd.DataFrame([date for date in (immig_nmf[14].values)], index=(immig_nmf[0].values))
immig_sentiment_pos = immig_sentiment.groupby(immig_sentiment[0]).get_group(1)
immig_sentiment_neg = immig_sentiment.groupby(immig_sentiment[0]).get_group(0)
immig_sentiment_pos = (pd.DataFrame.from_dict(Counter(immig_sentiment_pos[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')
immig_sentiment_neg = (pd.DataFrame.from_dict(Counter(immig_sentiment_neg[0].index.str[:7]), orient='index').reset_index()).sort_values(by=['index']).set_index('index')

ax = plt.gca()

resist_sentiment_pos.plot(style=[':', '--', '-'], color='blue', ax=ax)
resist_sentiment_neg.plot(style=[':', '--', '-'], color='red', ax=ax)
plt.ylabel('# Posts per Month')
plt.xlabel('Time-Series')
plt.title('Immigration NMF Sentiment Analysis')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.plot(race_sentiment_neg[0].index,
        race_sentiment_neg[0], 
        linestyle=":", 
        color='red')
ax.plot(race_sentiment_pos[0].index,
        race_sentiment_pos[0],
        color='blue',
        linestyle="--")
ax.set(xlabel="Date",
       ylabel="Negative Sentiment",
       title="Daily Total Precipitation\nBoulder, Colorado in July 2018")
plt.gcf().autofmt_xdate()
ax.xaxis.set_major_locator(plt.MaxNLocator(30))
plt.show()

In [ ]:
immig_sentiment_pos.index